In [8]:
pip install -r ../requirements.txt


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install seaborn


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install -q plotly
!pip install -q folium


In [11]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

In [12]:
url='https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv'
df=pd.read_csv(url, sep=',')
df.shape

(891, 12)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Como existe un desabalance me tengo que fijar mas en las medidas de f1 y no tanto en accuracy

In [14]:
df['Survived'].value_counts()/len(df['Survived'])

0    0.616162
1    0.383838
Name: Survived, dtype: float64

Decision tree, logistic regression, entre otros. 
Algoritmo mas complejos se necesitan mas datos. 
Criterio para analizar random forest es mas complejo para interpretar, deberiamos de analizar cada arbol y realizar el promedio.


In [15]:
df=df.drop(columns=['Name'])

In [16]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [17]:
df['Sex']=df['Sex'].map({'male':1,'female':0})
df['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [18]:
df['Ticket'].nunique()

681

In [51]:
df


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.000000,1,0,7.2500,2.0
1,2,1,1,0,38.000000,1,0,71.2833,1.0
2,3,1,3,0,26.000000,0,0,7.9250,2.0
3,4,1,1,0,35.000000,1,0,53.1000,2.0
4,5,0,3,1,35.000000,0,0,8.0500,2.0
...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,13.0000,2.0
887,888,1,1,0,19.000000,0,0,30.0000,2.0
888,889,0,3,0,29.699118,1,2,23.4500,2.0
889,890,1,1,1,26.000000,0,0,30.0000,1.0


In [19]:
df=df.drop(columns=['Ticket'])

In [20]:
df['Cabin'].nunique()
df=df.drop(columns=['Cabin'])

In [21]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [22]:
df['Embarked']=df['Embarked'].map({'S':2,'C':1,'Q':0})
df['Embarked']

0      2.0
1      1.0
2      2.0
3      2.0
4      2.0
      ... 
886    2.0
887    2.0
888    2.0
889    1.0
890    0.0
Name: Embarked, Length: 891, dtype: float64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


In [24]:
df['Age'][np.isnan(df['Age'])]=df['Age'].mean()


/tmp/ipykernel_22790/211817231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][np.isnan(df['Age'])]=df['Age'].mean()


In [25]:
df['Embarked'].value_counts()

2.0    644
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [26]:
df['Embarked'][np.isnan(df['Embarked'])]=2

/tmp/ipykernel_22790/960493807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])]=2


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


In [28]:
X=df.drop(columns=['Survived','PassengerId','Sex','Embarked'])
y=df['Survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=520, test_size=0.2)

In [29]:
modelo=RandomForestClassifier(n_estimators=50,random_state=145)

In [30]:
modelo.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50, random_state=145)

In [31]:
y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [32]:
y.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [33]:
train_report=metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       456
           1       0.96      0.92      0.94       256

    accuracy                           0.96       712
   macro avg       0.96      0.95      0.95       712
weighted avg       0.96      0.96      0.96       712



In [34]:
test_report=metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.66      0.87      0.75        93
           1       0.79      0.52      0.63        86

    accuracy                           0.70       179
   macro avg       0.73      0.70      0.69       179
weighted avg       0.72      0.70      0.69       179



f1 score del train es 1 y del test es 0.76, el algoritmo esta memorizando la data. 
estaria mas conforme con un train .78 y test.76

In [35]:
modelo=RandomForestClassifier(n_estimators=5,random_state=42,max_depth=5)

In [36]:
modelo.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=5, random_state=42)

In [37]:

y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [38]:
train_report=metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.79      0.89      0.83       456
           1       0.74      0.58      0.65       256

    accuracy                           0.78       712
   macro avg       0.76      0.73      0.74       712
weighted avg       0.77      0.78      0.77       712



In [39]:
test_report=metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.61      0.80      0.69        93
           1       0.67      0.45      0.54        86

    accuracy                           0.63       179
   macro avg       0.64      0.62      0.62       179
weighted avg       0.64      0.63      0.62       179



In [40]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
#Criterio
criterion=['gini','entropy']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
#'max_features': max_features, # Son muy pocas variables por lo cual no vale la pena aplicarlo
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap,
'criterion':criterion}
print(random_grid)

{'n_estimators': [200, 233, 266, 300, 333, 366, 400, 433, 466, 500], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']}


In [41]:
classif_grid=RandomForestClassifier(random_state=1107)
classif_grid_random=RandomizedSearchCV(estimator=classif_grid,n_iter=100,cv=5,random_state=1107,param_distributions=random_grid)

In [42]:
#classif_grid_random.fit(X_train,y_train)

In [43]:
#classif_grid_random.best_params_

In [44]:
modelo_final=RandomForestClassifier(n_estimators= 333, min_samples_split= 10,min_samples_leaf=1,max_depth= 20,criterion='gini',
bootstrap= True)

In [45]:
modelo_final.fit(X_train, y_train)
y_train_final=modelo_final.predict(X_train)
y_test_final=modelo_final.predict(X_test)

In [46]:
train_report_final=metrics.classification_report(y_pred=y_train_final,y_true=y_train)
print(train_report_final)

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       456
           1       0.84      0.75      0.79       256

    accuracy                           0.86       712
   macro avg       0.85      0.84      0.84       712
weighted avg       0.86      0.86      0.86       712



In [47]:
train_report_final=metrics.classification_report(y_pred=y_test_final,y_true=y_test)
print(train_report_final)

              precision    recall  f1-score   support

           0       0.66      0.87      0.75        93
           1       0.79      0.51      0.62        86

    accuracy                           0.70       179
   macro avg       0.72      0.69      0.68       179
weighted avg       0.72      0.70      0.69       179



In [48]:
modelo_final

RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=333)

In [49]:
import pickle

filename = '../models/titanic_model.pickle'
pickle.dump(modelo_final, open(filename, 'wb'))

In [50]:
X_train.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')